In [1]:
import pandas as pd
import sqlite3
import json

In [2]:
conn = sqlite3.connect('music.db')

# Análise das TOP TAGS

A primeira da lista que vem do Last.FM, no máximo uma por track

In [3]:
top_tags = """
SELECT
    *
FROM
    user_data
INNER JOIN
    artists
    ON user_data.artist_name = artists.name
INNER JOIN
    tracks
    ON user_data.track_name = tracks.name AND artists.mbid = tracks.artist_mbid
INNER JOIN
    top_track_tags ON tracks.mbid = top_track_tags.track_mbid
INNER JOIN
    tags ON top_track_tags.tag_id = tags.rowid
"""

top_tags = pd.read_sql(top_tags, con=conn)
print(top_tags.columns)

Index(['end_time', 'artist_name', 'track_name', 'ms_played', 'mbid', 'name',
       'mbid', 'name', 'artist_mbid', 'album_mbid', 'listeners', 'duration',
       'playcount', 'updated_on', 'date_published', 'track_mbid', 'tag_id',
       'tag_name'],
      dtype='object')


In [4]:
# Top tags only
top_tags.groupby(['tag_name'])['ms_played'].sum().sort_values(ascending=False)[:20]

tag_name
classic rock        2930238627
rock                1138723622
Progressive rock     655285904
hard rock            352425204
80s                  328215429
indie rock           190910463
blues                171327540
jazz                  83507069
Southern Rock         72987943
instrumental          60402814
Hip-Hop               55534174
indie                 49761958
classical guitar      41939912
alternative rock      40542370
heavy metal           37341282
rap                   35538280
country rock          34669296
alternative           30153013
latin                 26159448
blues rock            25491278
Name: ms_played, dtype: int64

In [30]:
# Aqui eu dou left em tudo, pq tem muito artista que não achou na LASTFM então nao tem entry nas tabelas artists/tracks
missing_top_tags = """
SELECT
    *
FROM
    user_data
LEFT JOIN
    artists
    ON user_data.artist_name = artists.name
LEFT JOIN
    tracks
    ON user_data.track_name = tracks.name AND artists.mbid = tracks.artist_mbid
LEFT JOIN
    top_track_tags ON tracks.mbid = top_track_tags.track_mbid
LEFT JOIN
    tags ON top_track_tags.tag_id = tags.rowid
"""

missing_top_tags = pd.read_sql(missing_top_tags, con=conn)
print(missing_top_tags.columns)

Index(['end_time', 'artist_name', 'track_name', 'ms_played', 'mbid', 'name',
       'mbid', 'name', 'artist_mbid', 'album_mbid', 'listeners', 'duration',
       'playcount', 'updated_on', 'date_published', 'track_mbid', 'tag_id',
       'tag_name'],
      dtype='object')


In [31]:
missing_top_tags['tag_name'].isnull().sum()

21589

In [32]:
# O quanto de tempo escutado (EM HORAS) sem tag que eu estaria deixando de analisar.
missing_top_tags.loc[missing_top_tags['tag_name'].isna()]['ms_played'].sum() * 2.77778e-7

980.302203963358

In [33]:
# O quanto de tempo escutado (EM HORAS) eu estou analisando
missing_top_tags.loc[missing_top_tags['tag_name'].notna()]['ms_played'].sum() * 2.77778e-7

1853.052313551776

> Realmente, o numero de musicas que nao pegou no LAST FM é bem grande

# Análise das musicas mais novas

In [35]:
# Aqui vou dar left join em ARTISTS pra ver alguns artistas mais recentes que ouvi
new_songs = """
SELECT
    end_time, artist_name, track_name, ms_played, artists.name AS artistName, tracks.name as trackName, tags.tag_name
FROM
    user_data
LEFT JOIN
    artists
    ON user_data.artist_name = artists.name
LEFT JOIN
    tracks
    ON user_data.track_name = tracks.name AND artists.mbid = tracks.artist_mbid
LEFT JOIN
    top_track_tags ON tracks.mbid = top_track_tags.track_mbid
LEFT JOIN
    tags ON top_track_tags.tag_id = tags.rowid
ORDER BY end_time DESC
LIMIT 500
"""

new = pd.read_sql(new_songs, con=conn)
new[50:75]

,end_time,artist_name,track_name,ms_played,artistName,trackName,tag_name
50,2021-09-03 14:31:00,Joshua Morse,Fairy Dust,166229,Joshua Morse,None,None
51,2021-09-03 14:28:00,Jokabi,Fairy Fountain,169285,None,None,None
52,2021-09-03 14:25:00,Mezhdunami,Bedtime Stories,191389,None,None,None
53,2021-09-03 14:25:00,Jonas Munk Lindbo,Song of Storms - Lofi Mix,35790,None,None,None
54,2021-09-03 14:22:00,Mezhdunami,Warm,315075,None,None,None
55,2021-09-03 14:16:00,Gibarian,Yearning,320250,None,None,None
56,2021-09-03 14:11:00,oDDling,Make Believe,196010,None,None,None
57,2021-09-03 14:08:00,Cinematix1993,Dreaming in Memories,163000,None,None,None
58,2021-09-03 14:02:00,Tonebox,Dimensional,308600,None,None,None
59,2021-09-03 13:57:00,Mezhdunami,Ice Caves,159754,None,None,None


Percebi que tem muita musica que não tava pegando nos queries de antes pq com o artista é pequeno ele não tinha um perfil no lastFM. Assim, muita musica não apareceu lá.

O que vou precisar fazer pra pegar esses artistas menores é classificar à mão. Eu posso tentar generalizar o genero por artista que não tenha perfil também.

Talvez vale a pena tentar um novo API de dados.

# Classificando dados manuais

Testei APIs diferente para as músicas porém nenhum consegue pegar os artistas de LoFi com o genero. Assim, o genero vai ter que ser feito manualmente. A ideia de classificar a partir das playlists não funciona pq o spotify só informa os dados das playlist CRIADAS pelo usuário e não pelas que ele se inscreve/escuta. 

O que eu vou fazer é agrupar as músicas pelo ARTISTA e dar sort pelos mais escutados. Assim, de artista em artista eu vou classificar o gênero usando o app Label Studio pra fazer ficar mais fácil.

As músicas que tenho os tags ainda vou precisar fazer uma classificação manual também para agrupar os gêneros.

## Musicas sem tags

In [37]:
tagless = """
SELECT
    artist_name,
    track_name,
    ms_played
FROM
    user_data
LEFT JOIN
    artists
    ON user_data.artist_name = artists.name
LEFT JOIN
    tracks
    ON user_data.track_name = tracks.name AND artists.mbid = tracks.artist_mbid
LEFT JOIN
    top_track_tags ON tracks.mbid = top_track_tags.track_mbid
LEFT JOIN
    tags ON top_track_tags.tag_id = tags.rowid
WHERE tags.tag_name IS NULL
"""
tagless = pd.read_sql(tagless, con=conn)
tagless.head()

,artist_name,track_name,ms_played
0,Led Zeppelin,The Rover - 1993 Remaster,336906
1,Alex McArtor,Burning Fleeting Love,700
2,Alex McArtor,Burning Fleeting Love,85526
3,Coffee Date,Midna's Lament (Legend of Zelda),8778
4,Mikel,Fairy Fountain,164266


In [40]:
tagless.groupby('artist_name')['ms_played'].sum().sort_values(ascending=False)[:20]

artist_name
Greta Van Fleet              271604291
Nujabes                      232611653
Mikel                        220106788
Led Zeppelin                 202258791
Blue Öyster Cult             137479612
Logic                        110906202
Pink Floyd                    90472395
Santana                       82364469
Chill Cow Lofi                78633897
Sabotage                      72981335
Fleetwood Mac                 57889958
Retro Remix Revue             54847002
GlitchxCity                   43612176
Deep Purple                   37157266
Boccherini Guitar Quartet     36187874
RZO                           30376848
Chris Christodoulou           29691119
Robert Westaway               27202728
Lofi Lia                      27156534
Lofi Hip Hop Nation           22995901
Name: ms_played, dtype: int64

## Top Tags

Aqui vou pegar as maiores tags e depois vou classificá-las em um número menor de nomes.

Exemplo: Classic Rock, Rock, Hard Rock -> Rock

In [90]:
top_tags = """
SELECT
    artist_name,
    track_name,
    ms_played,
    tag_name
FROM
    user_data
INNER JOIN
    artists
    ON user_data.artist_name = artists.name
INNER JOIN
    tracks
    ON user_data.track_name = tracks.name AND artists.mbid = tracks.artist_mbid
INNER JOIN
    top_track_tags ON tracks.mbid = top_track_tags.track_mbid
INNER JOIN
    tags ON top_track_tags.tag_id = tags.rowid
"""
top_tags = pd.read_sql(top_tags, con=conn)

In [91]:
# Top tags only
top_tags.groupby(['tag_name'])['ms_played'].sum().sort_values(ascending=False)[:20]

tag_name
classic rock        2930238627
rock                1138723622
Progressive rock     655285904
hard rock            352425204
80s                  328215429
indie rock           190910463
blues                171327540
jazz                  83507069
Southern Rock         72987943
instrumental          60402814
Hip-Hop               55534174
indie                 49761958
classical guitar      41939912
alternative rock      40542370
heavy metal           37341282
rap                   35538280
country rock          34669296
alternative           30153013
latin                 26159448
blues rock            25491278
Name: ms_played, dtype: int64

In [92]:
# Passando as top tags pra CSV, pra olhar por cima e decidir quais vão ser os generos que vou levar em conta na análise
# Passando pra horas também
(top_tags.groupby(['tag_name'])['ms_played'].sum().sort_values(ascending=False) * 2.77778e-7).to_frame().reset_index().to_csv('top_tags.csv', index=False)

- rock  
- blues  
- rap  
- electronic  
- LoFi  
- Bossa Nova  
- Pop  
- Jazz  
- Classical  

# Exportando dados pro Label Studio

## Top Tags

In [93]:
tag_data = (top_tags.groupby(['tag_name'])['ms_played'].sum().sort_values(ascending=False) * 2.77778e-7).to_frame().reset_index()
tag_data_export = []

dicts = tag_data.to_dict(orient='index')

for key, vals in dicts.items():
    tag_data_export.append({
        'item': vals
    })

In [94]:
with open('top_tags.json', 'w+') as f:
    json.dump(tag_data_export, f)
    

## Unfound tracks

In [95]:
no_tags = """
SELECT
    end_time, artist_name, track_name, ms_played, artists.name AS artistName, tracks.name as trackName, tags.tag_name
FROM
    user_data
LEFT JOIN
    artists
    ON user_data.artist_name = artists.name
LEFT JOIN
    tracks
    ON user_data.track_name = tracks.name AND artists.mbid = tracks.artist_mbid
LEFT JOIN
    top_track_tags ON tracks.mbid = top_track_tags.track_mbid
LEFT JOIN
    tags ON top_track_tags.tag_id = tags.rowid
WHERE tags.tag_name IS NULL
"""

no_tags = pd.read_sql(no_tags, con=conn)
no_tags.head()

,end_time,artist_name,track_name,ms_played,artistName,trackName,tag_name
0,2020-10-03 03:45:00,Led Zeppelin,The Rover - 1993 Remaster,336906,Led Zeppelin,None,None
1,2020-10-03 17:29:00,Alex McArtor,Burning Fleeting Love,700,None,None,None
2,2020-10-03 17:40:00,Alex McArtor,Burning Fleeting Love,85526,None,None,None
3,2020-10-03 20:56:00,Coffee Date,Midna's Lament (Legend of Zelda),8778,None,None,None
4,2020-10-03 20:59:00,Mikel,Fairy Fountain,164266,None,None,None


In [96]:
untagged_data = (no_tags.groupby(['artist_name'])['ms_played'].sum().sort_values(ascending=False) * 2.77778e-7).to_frame().reset_index()
untagged_data

,artist_name,ms_played
0,Greta Van Fleet,75.445697
1,Nujabes,64.614400
2,Mikel,61.140823
3,Led Zeppelin,56.183042
4,Blue Öyster Cult,38.188812
...,...,...
1052,Bob Dylan,0.000069
1053,Lindstrøm,0.000000
1054,Frankie Valli & The Four Seasons,0.000000
1055,Girolamo Giacomo Cocchi,0.000000


In [97]:
untagged_data_export = []

untag_dicts = untagged_data.to_dict(orient='index')

for key, vals in untag_dicts.items():
    untagged_data_export.append({
        'item': vals
    })

with open('untagged_artists.json', 'w+') as f:
    json.dump(untagged_data_export, f)
    